<a href="https://colab.research.google.com/github/priyasin01/OBJECT-DETECTION-CLASSIFICATION-AND-SEGMENTATION/blob/master/Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydrive
!git clone https://github.com/thtrieu/darkflow.git
!pip install Cython 
!pip install pytube
!pip install PyDrive

import os
os.chdir('/content/darkflow')
!pwd

!pip install -e .

    100% |████████████████████████████████| 993kB 19.4MB/s 
  Running setup.py bdist_wheel for pydrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive
Cloning into 'darkflow'...
remote: Enumerating objects: 2706, done.
remote: Total 2706 (delta 0), reused 0 (delta 0), pack-reused 2706
Receiving objects: 100% (2706/2706), 18.74 MiB | 7.79 MiB/s, done.
Resolving deltas: 100% (1781/1781), done.
    100% |████████████████████████████████| 2.1MB 12.3MB/s 
/content/darkflow
Obtaining file:///content/darkflow
  Running setup.py develop for darkflow


In [0]:
import pydrive
import time
from tqdm import tqdm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# load training weights

newpath = '/content/darkflow/bin'
if not os.path.exists(newpath):
  os.makedirs(newpath)

download = drive.CreateFile({'id': '1J-GZ6mvhu12j19zZtq9fTbnyJeCgxE0g'}) # insert the fild ID here
download.GetContentFile('/content/darkflow/bin/yolov2.weights')

In [0]:
from darkflow.net.build import TFNet
import cv2
import matplotlib.pyplot as plt

In [0]:
options = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolov2.weights',
    'threshold': 0.3,
    'gpu' : 1.0
          }

In [5]:
tfnet = TFNet(options)


/content/darkflow/darkflow/dark/darknet.py:54: UserWarning: ./cfg/yolov2.cfg not found, use cfg/yolo.cfg instead
  cfg_path, FLAGS.model))


Parsing cfg/yolo.cfg
Loading bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 0.11673593521118164s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bn

In [0]:
def predict_img(link):
  
  result = tfnet.return_predict(link)
   

  for n,value in enumerate(result):
    
    tl = (result[n]['topleft']['x'], result[n]['topleft']['y'])
    br = (result[n]['bottomright']['x'], result[n]['bottomright']['y'])
    label = result[n]['label']
    img = cv2.rectangle(img,tl,br,(0,255,0))
    
  plt.figure(figsize=(20,10))    
  plt.imshow(img)

In [0]:


def create_image_paths():
  
  import glob
  # Get all image file paths to a list.
  images = list(glob.iglob(os.path.join('/content/sample_data_frames/',title, '*.*')))
  # Sort the images by name index.
  images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
  
  return images



In [0]:
def download_youtube_video(link):
  
  
  from pytube import YouTube
  yt = YouTube(link)
  streams = yt.streams.get_by_itag('18')
  streams.download('/content/sample_data')
  title = streams.default_filename
  
  print('Youtube Video Downloaded...')
  return  title

In [0]:
def fps():
  
  
  video = cv2.VideoCapture(os.path.join('/content/sample_data/',title))  
  fps = video.get(cv2.CAP_PROP_FPS)
  print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
  video.release()
  return fps

In [0]:
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from tqdm import tqdm
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in tqdm(images):
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid


In [0]:
def sample_frames():
  
  capture = cv2.VideoCapture(os.path.join('/content/sample_data/',title)) 

  
  frames = []
  frame_count = 0

  video_frame_path = os.path.join('/content/sample_data_frames/',title)
  
  if not os.path.exists(video_frame_path):
    os.makedirs(video_frame_path)
    print('Frames Directory Created...')
  
  while True:
  
    ret, frame = capture.read()
    frames.append(frame)
    frame_count += 1
    
  
  
    if not ret:
      print(frame_count)
      return (frames,frame_count,video_frame_path)
      break

In [0]:
def predict_video(link):
  
  global title
  
  title = download_youtube_video(link)
   
  frames,frame_count,video_frame_path = sample_frames()
  
  #load video and get every frame of it for analysis..
  

  
  # for every frame run yolo prediction and save the frame as jpg file

  for count,frame in tqdm(enumerate(frames[1:-10])):
    

    #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    results = tfnet.return_predict(frame)
  
    for n,value in enumerate(results):
      
      tl = (results[n]['topleft']['x'], results[n]['topleft']['y'])
      br = (results[n]['bottomright']['x'], results[n]['bottomright']['y'])
      label = results[n]['label']
      frame = cv2.rectangle(frame,tl,br,(0,255,0),3)
      frame = cv2.putText(frame,label,tl,cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
      
      #frame = cv2.addText(label)
      #frame = cv2.line(img,(0,frame.shape[0]//2),(600,img.shape[0]//2),(255,0,0),2)   
      
  
    name = '{0}.jpg'.format(count)
    
    name = os.path.join(video_frame_path,name)  
    cv2.imwrite(name, frame)
    
  print('All Frames Saved...')
  
  newpath = '/content/sample_data_out/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
    
  images = create_image_paths()
  #print(images)
  frames_per_second = fps()
  make_video(os.path.join('/content/sample_data_out/',title), images, fps=frames_per_second)

In [14]:
predict_video('https://www.youtube.com/watch?v=wlJzRAUexV0')

Youtube Video Downloaded...
Frames Directory Created...


0it [00:00, ?it/s]

1939


1928it [02:33, 12.57it/s]
  1%|          | 14/1928 [00:00<00:13, 137.13it/s]

All Frames Saved...
Frames per second using video.get(cv2.CAP_PROP_FPS) : 29.87667009249743


100%|██████████| 1928/1928 [00:10<00:00, 185.74it/s]


In [0]:


files.download(os.path.join('/content/sample_data_out/',title))

